In [1]:
%display latex

# Manifolds

In this notebook is an abridged version of a full [SageManifolds tutorial](https://nbviewer.jupyter.org/github/sagemanifolds/SageManifolds/blob/master/Notebooks/SM_tutorial.ipynb), with my own additions in places.

It serves as a reference for myself, and rewritten as a way of reminding myself of the manifold features.

<!--BEGIN TOC-->
## Table of Contents
1. [Basic manifold operations](#0-Basic-manifold-operations)
    1. [Defining charts](#0-Defining-charts)
    2. [Functions on charts](#0-Functions-on-charts)
    3. [Charts on open subsets](#0-Charts-on-open-subsets)
2. [Chart operations](#0-Chart-operations)
    1. [Transition maps](#0-Transition-maps)
    2. [Plotting charts](#0-Plotting-charts)
3. [Points on manifolds](#0-Points-on-manifolds)

<!--END TOC-->

## Basic manifold operations <a name="0-Basic-manifold-operations"></a>
We start by defining a manifold, which by default is a *differentiable* manifold over $\mathbb{R}^3$:

- the first argument is the **dimensionality** of the manifold
- the second argument is the **name** of the manifold; it is the mathematical symbol which will represent the manifold in our computations

We then also use the `latex_name` keyword argument to set the display symbol of the manifold.

In [2]:
M = Manifold(3, 'M', latex_name=r"\mathcal{M}")

By default, the indexes in SageManifold start at 0, however the `Manifold` constructor provides the keyword argument `start_index` if an alternative start is desireable.

The default field of the manifold is always $\mathbb{R}$, however alternatives exist. From the [documentation](https://doc.sagemath.org/html/en/reference/manifolds/sage/manifolds/manifold.html#sage.manifolds.manifold.Manifold) for `Manifold`, we see any class inheriting from `Fields` or `TopologicalSpaces` may be used as the manifold's field.

Also note that two string arguments are defined:
- `complex` for $\mathbb{C}$
- `real` (default) for $\mathbb{R}$

The documentation also describes the `structure` keyword, which can take a number of different string arguments: common uses are
- `smooth` (default)
- `Riemannian` for real differentiable manifold with a positive definite metric
- `Lorentzian` for real differentiable manifold with Lorentzian metric, where the signature may be provided via the keyword `signature` as either `positive` (default) or `negative`.

To view the [category](https://doc.sagemath.org/html/en/reference/categories/sage/categories/category.html?highlight=category#module-sage.categories.category) of our manifold, we use:

In [3]:
category(M)

Category of smooth manifolds over Real Field with 53 bits of precision

We can view the indices using the `irange()` method:

In [4]:
[i for i in M.irange()]

[0, 1, 2]

*NB:* the above returns a generator.

### Defining charts <a name="0-Defining-charts"></a>
A [coordinate chart](https://en.wikipedia.org/wiki/Atlas_(topology)#Charts) of a manifold is a homeomorphism $\phi$ from an open-subset $U \subseteq M$ to an open-subset of a Euclidean space, i.e. a coordinate space. For example:
$$
\phi : U \rightarrow \mathbb{R}^n
$$

A chart is denoted by the ordered pair $(U, \phi)$.

Suppose $\mathcal{M}$ can be covered my a single chart ($U \sim M$); we may define the chart using the angle bracket `<coordinates>` short-hand:

In [5]:
X.<x,y,z> = M.chart()

By default, the `chart` method assumes the coordinate range of each coordinate is $(-\infty,+\infty)$. In the [Coordinate Chart documentation](https://doc.sagemath.org/html/en/reference/manifolds/sage/manifolds/chart.html?highlight=chart#module-sage.manifolds.chart) are provided ways of applying further restrictions to a chart.

The notation for restricting the coordinates is:
```py
r"symbol:(limits):latexsymbol"
```
where the limits and latexsymbol are optional. Thus, our above definition is equivalent to
```py
M.chart("x:(-inf,inf):x y z")
```
We can perform basic operations on the chart:

In [6]:
X

Chart (M, (x, y, z))

In [7]:
X[:]

(x, y, z)

In [8]:
X[0] is x

True

Coordinates are SageMath `Expressions`, and thus provide the same interface. Creating more complicated structres in then straight forward:

### Functions on charts <a name="0-Functions-on-charts"></a>

We define functions on the chart's [codomain](https://en.wikipedia.org/wiki/Codomain); in our case, functions are
$$
f: \mathbb{R}^3 \rightarrow \mathbb{R}
$$

In [9]:
f = X.function(x + y^2 + z^3)
f.display()

(x, y, z) |--> z^3 + y^2 + x

Which can be evaluated with

In [10]:
f(5, 3, 7)

357

Functions on charts in SageManifold are an instance of the `ChartFunction` type; they perform automatic simplifications in all operations. 

To illustrate this, consider two regular symbolic functions:

In [11]:
f0(x, y, z) = cos(x)^2
g0(x, y, z) = sin(x)^2

f0 + g0

(x, y, z) |--> cos(x)^2 + sin(x)^2

However, the same as `ChartFunction`s behaves as

In [12]:
f1 = X.function(cos(x)^2)
g1 = X.function(sin(x)^2)

(f1 + g1).display()

(x, y, z) |--> 1

which is the equivalent of using `simplify_trig()` on the symbolic functions:

In [13]:
(f0 + g0).simplify_trig()

(x, y, z) |--> 1

We can convert a ChartFunction to a symbolic expression through the `expr()` member function:

In [14]:
f1.expr()

cos(x)^2

### Charts on open subsets <a name="0-Charts-on-open-subsets"></a>
Consider an open-subset of $\mathcal{M}$, for example the complement $U$ of the region defined ${y=0, x \geq 0}$.

In SageMath notation, the brackets 
- `(y!=0, x<0)` would read $ y \neq 0$ *or* $x<0$, whereas 
- `[y!=0, x<0]` would read $ y \neq 0$ *and* $x<0$.

We define such a $U$ using the `open_subset()` method:

In [15]:
U = M.open_subset(
    'U', 
    coord_def={X: (y!=0, x<0)}
)

We call `X_U` the restriction of the chart $X$ on $U$:

In [16]:
X_U = X.restrict(U)
X_U

Chart (U, (x, y, z))

## Chart operations <a name="0-Chart-operations"></a>
Let us define a new chart on $U$ with spherical coordinates:

In [17]:
coords = r"r:(0,+oo) theta:(0,pi):\theta phi:(0,2*pi):\phi"
Y.<r,theta,phi> = U.chart(coords)

Y

Chart (U, (r, theta, phi))

At any point, we can view the assumptions that sage is currently making:

In [18]:
for assumpt in assumptions():
    print(assumpt)

x is real
y is real
z is real
r is real
r > 0
theta is real
theta > 0
theta < pi
phi is real
phi > 0
phi < 2*pi


These are used seemlessly in simplifications, such as:

In [19]:
simplify(abs(r)), simplify(abs(x))

(r, abs(x))

Note that $r$ was simplified, as the assumption is $r>0$.

### Transition maps <a name="0-Transition-maps"></a>
We can define transition maps between charts. For example, declaring a transition map from `Y` to `X_U` may be done with:

In [20]:
xfm_Y2X_U = Y.transition_map(
    X_U,
    [
        r*sin(theta)*cos(phi), 
        r*sin(theta)*sin(phi), 
        r*cos(theta)
    ]
)
xfm_Y2X_U

Change of coordinates from Chart (U, (r, theta, phi)) to Chart (U, (x, y, z))

In [21]:
xfm_Y2X_U.display()

x = r*cos(phi)*sin(theta)
y = r*sin(phi)*sin(theta)
z = r*cos(theta)

Given a forward transformation, SageManifold can sometimes calculate the inverse for us by using
```py
xfm_Y2X_U.inverse()
```

However, in known systems, such as the above, it may be simpler to provide a system of equations and allow SageManifold to check it for us:

In [22]:
xfm_Y2X_U.set_inverse(
    sqrt(x^2+y^2+z^2), 
    atan2(sqrt(x^2+y^2),z), 
    atan2(y, x)
)

Check of the inverse coordinate transformation:
  r == r  *passed*
  theta == arctan2(r*sin(theta), r*cos(theta))  **failed**
  phi == arctan2(r*sin(phi)*sin(theta), r*cos(phi)*sin(theta))  **failed**
  x == x  *passed*
  y == y  *passed*
  z == z  *passed*
NB: a failed report can reflect a mere lack of simplification.


The failure for the `arctan2` functions is indeed related to lack of simplification.

We can all the same proceed:

In [23]:
xfm_Y2X_U.inverse().display()

r = sqrt(x^2 + y^2 + z^2)
theta = arctan2(sqrt(x^2 + y^2), z)
phi = arctan2(y, x)

At any point, we can view the *user's* atlas (not a maximal atlas):

In [24]:
M.atlas()

[Chart (M, (x, y, z)), Chart (U, (x, y, z)), Chart (U, (r, theta, phi))]

The first chart in the atlas is considered the *default* chart when computing operations on the manifold. This can be updated by using the `set_default_chart()` method.

We can view the default chart explicitly with:

In [25]:
M.default_chart()

Chart (M, (x, y, z))

The open-subset has its own atlas, which will be a subset of the manifold's atlas:

In [26]:
U.atlas()

[Chart (U, (x, y, z)), Chart (U, (r, theta, phi))]

### Plotting charts <a name="0-Plotting-charts"></a>
We can draw one chart in terms of the other using the convenient `.plot()` method.

For example, plotting `Y` in terms of the chart of `X` (that is, the lines of constant coordinates in Y drawn over the coordinates of X):

In [27]:
Y.plot(X)

Graphics3d Object

*NB:* that this is identical to
```py
Y.plot(X_U)
```
since X_U is merely a restriction of X. The transition map defined onto the restriction is trivially extended.

The [plot command](https://doc.sagemath.org/html/en/reference/manifolds/sage/manifolds/chart.html#sage.manifolds.chart.RealChart.plot) has a huge number of options for controlling how the plot looks.

For example:

In [28]:
Y.plot(
    X,
    ranges={r:(0,2), theta:(0,pi/2)},
    number_values=5,
    color={r:"blue", theta:"green", phi:"red"},
    aspect_ratio=1
)

Graphics3d Object

We may also plot `X_U` in terms of `Y`, although as the domain of `X_U` is larger, we must limit ourselves to a restriction:

In [29]:
graph = X_U.plot(Y)
show(graph, axes_labels=["r", "theta", "phi"])

Graphics3d Object

## Points on manifolds <a name="0-Points-on-manifolds"></a>
A point $p$ on $\mathcal{M}$ is defined through coordinates of a chart:

In [30]:
p = M.point((1, 2, -1), name='p')
print(p)

Point p on the 3-dimensional differentiable manifold M


The points exist, by default on the manifold's default chart. To specify a different chart, we can pass the
```py
chart=X
```
keyword to the `.point()` method.

We can then check some basic properties of our point:

In [31]:
p in M, p in U, p in X

(True, True, False)

*NB:* $p \in U$ by virtue of $y \neq 0$.

The last may seem a little surprising, but this is just syntactic, since `X` is $(\mathcal{M}, (x, y, z))$, we would instead write:

In [32]:
p in X.domain()

True

Consider now a point $q$ with $y = 0$ and $x \geq 0$:

In [33]:
q = M.point((1, 0, 2), name="q")

q in U

False

We can read the coordinates of a point using the `.coord()` method:

In [34]:
p.coord()

(1, 2, -1)

Note again, this is using the default chart of $\mathcal{M}$. We can specify an alternative chart by passing it as an argument:

In [35]:
p.coord(X_U), p.coord(Y)

((1, 2, -1), (sqrt(3)*sqrt(2), pi - arctan(sqrt(5)), arctan(2)))

We can also act the point on a chart, as alternative syntax:

In [36]:
X(p)

(1, 2, -1)

Which is exactly the mathematical operation of a chart. 

Note that if the point lies outside of a given chart, this operation throws an error:

In [37]:
try:
    X_U(q)
except ValueError as e:
    print(f"Error: {str(e)}")

Error: the point does not belong to the domain of Chart (U, (x, y, z))


We can also compare points:

In [38]:
p == q

False

This operation will implicitly use transition maps we may have defined:

In [39]:
p1 = U.point(Y(p), chart=Y)

p1 == p

True

In the language of SageManifold, these points are *elements* whose *parents* are the manifolds on which they have been defined; we may view:

In [40]:
p.parent(), q.parent(), p1.parent()

(3-dimensional differentiable manifold M,
 3-dimensional differentiable manifold M,
 Open subset U of the 3-dimensional differentiable manifold M)